# Use a federated learning strategy

Welcome to the next part of the federated learning tutorial. In previous parts of this tutorial, we introduced federated learning with PyTorch and Flower ([part 1](https://flower.ai/docs/framework/tutorial-get-started-with-flower-pytorch.html)).

In this notebook, we'll begin to customize the federated learning system we built in the introductory notebook (again, using [Flower](https://flower.ai/) and [PyTorch](https://pytorch.org/)).

> [Star Flower on GitHub](https://github.com/adap/flower) ⭐️ and join the Flower community on Slack to connect, ask questions, and get help: [Join Slack](https://flower.ai/join-slack) 🌼 We'd love to hear from you in the `#introductions` channel! And if anything is unclear, head over to the `#questions` channel.

Let's move beyond FedAvg with Flower strategies!

## Preparation

Before we begin with the actual code, let's make sure that we have everything we need.

### Installing dependencies

First, we install the necessary packages:

In [1]:
!pip install -q flwr[simulation] torch torchvision

zsh:1: no matches found: flwr[simulation]


Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [2]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import flwr as fl

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cpu using PyTorch 2.2.1 and Flower 1.9.0


It is possible to switch to a runtime that has GPU acceleration enabled (on Google Colab: `Runtime > Change runtime type > Hardware acclerator: GPU > Save`). Note, however, that Google Colab is not always able to offer GPU acceleration. If you see an error related to GPU availability in one of the following sections, consider switching back to CPU-based execution by setting `DEVICE = torch.device("cpu")`. If the runtime has GPU acceleration enabled, you should see the output `Training on cuda`, otherwise it'll say `Training on cpu`.

### Data loading

Let's now load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set), and wrap everything in their own `DataLoader`. We introduce a new parameter `num_clients` which allows us to call `load_datasets` with different numbers of clients.

In [3]:
NUM_CLIENTS = 10


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


### Model training/evaluation

Let's continue with the usual model definition (including `set_parameters` and `get_parameters`), training and test functions:

In [4]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

### Flower client

To implement the Flower client, we (again) create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`. Here, we also pass the `cid` to the client and use it log additional details:

In [5]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader).to_client()

## Strategy customization

So far, everything should look familiar if you've worked through the introductory notebook. With that, we're ready to introduce a number of new features. 

### Server-side parameter **initialization**

Flower, by default, initializes the global model by asking one random client for the initial parameters. In many cases, we want more control over parameter initialization though. Flower therefore allows you to directly pass the initial parameters to the Strategy:

In [8]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# Configure the server for just 3 rounds of training
config = fl.server.ServerConfig(num_rounds=3)

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

# Specify client resources via the backend config if you need GPU (defaults to 2 CPUs and 0 GPU)
backend_config = {"client_resources": None}
if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_gpus": 1}}

# Create an instance of the ClientApp
client_app = fl.client.ClientApp(client_fn=client_fn)

# Create an instance of the ServerApp with the config and strategy defined above
server_app = fl.server.ServerApp(config=config, strategy=strategy)

fl.simulation.run_simulation(
    server_app=server_app,
    client_app=client_app,
    backend_config=backend_config,
    num_supernodes=NUM_CLIENTS,    
)

INFO :      Flower ECE: Starting Driver API (gRPC-rere) on 0.0.0.0:9091
INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      Registered 10 nodes
INFO :      
INFO :      Supported backends: ['ray']
INFO :      [INIT]
INFO :      Initialising: RayBackend
INFO :      DriverServicer.CreateRun
INFO :      Using initial global parameters provided by strategy
INFO :      Backend config: {'client_resources': None}
INFO :      Evaluating initial global parameters
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)
2024-04-19 15:31:02,457	INFO worker.py:1621 -- Started a local Ray instance.
INFO :      `client_resources` not specified in backend config. Applying default setting: {'num_cpus': 2, 'num_gpus': 0.0}
INFO :      Constructed ActorPool with: 5 actors


(ClientAppActor pid=3070) [Client 0] fit, config: {}
(ClientAppActor pid=3070) Epoch 1: train loss 0.06496372818946838, accuracy 0.2228888888888889


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=3073) [Client 0] evaluate, config: {}


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


...

INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=3071) [Client 9] evaluate, config: {} [repeated 2x across cluster]


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 rounds in 21.71s
INFO :      History (loss, distributed):
INFO :      	('\tround 1: 0.06274059613545736\n'
INFO :      	 '\tround 2: 0.05564652236302694\n'
INFO :      	 '\tround 3: 0.052179469903310134\n')
INFO :      


(ClientAppActor pid=3070) [Client 2] fit, config: {} [repeated 2x across cluster]
(ClientAppActor pid=3070) Epoch 1: train loss 0.053701214492321014, accuracy 0.3626666666666667 [repeated 3x across cluster]
(ClientAppActor pid=3072) [Client 9] evaluate, config: {} [repeated 3x across cluster]


INFO :      Terminated RayBackend
INFO :      Stopping Simulation Engine now.


Passing `initial_parameters` to the `FedAvg` strategy prevents Flower from asking one of the clients for the initial parameters. If we look closely, we can see that the logs do not show any calls to the `FlowerClient.get_parameters` method.

### Starting with a customized strategy

We've seen the function `run_simulation` before. It accepts a number of arguments, amongst them the `server_app` which wraps around the strategy and number of training rounds, `client_app` which wraps around the `client_fn` used to create `FlowerClient` instances, and the number of clients to simulate which equals `num_supernodes`.

The strategy encapsulates the federated learning approach/algorithm, for example, `FedAvg` or `FedAdagrad`. Let's try to use a different strategy this time:

In [7]:
# Create FedAdagrad strategy
strategy = fl.server.strategy.FedAdagrad(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
)

# Create an instance of the ServerApp with the config and strategy defined above
server_app = fl.server.ServerApp(config=config, strategy=strategy)

# Start simulation
fl.simulation.run_simulation(
    server_app=server_app,
    client_app=client_app,
    backend_config=backend_config,
    num_supernodes=NUM_CLIENTS,    
)

INFO :      Flower ECE: Starting Driver API (gRPC-rere) on 0.0.0.0:9091
INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      Registered 10 nodes
INFO :      
INFO :      Supported backends: ['ray']
INFO :      [INIT]
INFO :      Initialising: RayBackend
INFO :      Using initial global parameters provided by strategy
INFO :      DriverServicer.CreateRun
INFO :      Backend config: {'client_resources': None}
INFO :      Evaluating initial global parameters
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)
2024-04-19 15:22:56,438	INFO worker.py:1621 -- Started a local Ray instance.
INFO :      `client_resources` not specified in backend config. Applying default setting: {'num_cpus': 2, 'num_gpus': 0.0}
INFO :      Constructed ActorPool with: 5 actors


(ClientAppActor pid=99298) [Client 1] fit, config: {}
(ClientAppActor pid=99298) Epoch 1: train loss 0.06442463397979736, accuracy 0.21466666666666667


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=99301) [Client 6] evaluate, config: {}


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


...

INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=99302) [Client 0] evaluate, config: {} [repeated 3x across cluster]
(ClientAppActor pid=99298) [Client 9] fit, config: {} [repeated 2x across cluster]
(ClientAppActor pid=99298) Epoch 1: train loss 0.14246177673339844, accuracy 0.15844444444444444 [repeated 3x across cluster]


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 rounds in 23.99s
INFO :      History (loss, distributed):
INFO :      	('\tround 1: 4.997446670532227\n'
INFO :      	 '\tround 2: 1.1978673082987468\n'
INFO :      	 '\tround 3: 0.13597038666407266\n')
INFO :      


(ClientAppActor pid=99300) [Client 9] evaluate, config: {} [repeated 2x across cluster]


INFO :      Terminated RayBackend
INFO :      Stopping Simulation Engine now.


## Server-side parameter **evaluation**

Flower can evaluate the aggregated model on the server-side or on the client-side. Client-side and server-side evaluation are similar in some ways, but different in others.

**Centralized Evaluation** (or *server-side evaluation*) is conceptually simple: it works the same way that evaluation in centralized machine learning does. If there is a server-side dataset that can be used for evaluation purposes, then that's great. We can evaluate the newly aggregated model after each round of training without having to send the model to clients. We're also fortunate in the sense that our entire evaluation dataset is available at all times.

**Federated Evaluation** (or *client-side evaluation*) is more complex, but also more powerful: it doesn't require a centralized dataset and allows us to evaluate models over a larger set of data, which often yields more realistic evaluation results. In fact, many scenarios require us to use **Federated Evaluation** if we want to get representative evaluation results at all. But this power comes at a cost: once we start to evaluate on the client side, we should be aware that our evaluation dataset can change over consecutive rounds of learning if those clients are not always available. Moreover, the dataset held by each client can also change over consecutive rounds. This can lead to evaluation results that are not stable, so even if we would not change the model, we'd see our evaluation results fluctuate over consecutive rounds.

We've seen how federated evaluation works on the client side (i.e., by implementing the `evaluate` method in `FlowerClient`). Now let's see how we can evaluate aggregated model parameters on the server-side:

In [9]:
# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net().to(DEVICE)
    valloader = valloaders[0]
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [10]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,  # Pass the evaluation function
)

# Create an instance of the ServerApp with the config and strategy defined above
server_app = fl.server.ServerApp(config=config, strategy=strategy)

# Start simulation
fl.simulation.run_simulation(
    server_app=server_app,
    client_app=client_app,
    backend_config=backend_config,
    num_supernodes=NUM_CLIENTS,
)

INFO :      Flower ECE: Starting Driver API (gRPC-rere) on 0.0.0.0:9091
INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      Registered 10 nodes
INFO :      
INFO :      Supported backends: ['ray']
INFO :      [INIT]
INFO :      Initialising: RayBackend
INFO :      Using initial global parameters provided by strategy
INFO :      DriverServicer.CreateRun
INFO :      Backend config: {'client_resources': None}
INFO :      Evaluating initial global parameters
INFO :      initial parameters (loss, other metrics): 0.07384053850173951, {'accuracy': 0.1}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.07384053850173951 / accuracy 0.1


2024-04-19 15:32:34,795	INFO worker.py:1621 -- Started a local Ray instance.
INFO :      `client_resources` not specified in backend config. Applying default setting: {'num_cpus': 2, 'num_gpus': 0.0}
INFO :      Constructed ActorPool with: 5 actors


(ClientAppActor pid=3712) [Client 0] fit, config: {}
(ClientAppActor pid=3712) Epoch 1: train loss 0.06482904404401779, accuracy 0.23222222222222222


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      fit progress: (1, 0.06361047673225403, {'accuracy': 0.258}, 6.3681846669642255)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


...

INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      fit progress: (3, 0.053451504945755006, {'accuracy': 0.37}, 19.14393854199443)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.053451504945755006 / accuracy 0.37
(ClientAppActor pid=3716) [Client 4] evaluate, config: {} [repeated 3x across cluster]
(ClientAppActor pid=3712) [Client 9] fit, config: {} [repeated 3x across cluster]


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 rounds in 22.59s
INFO :      History (loss, distributed):
INFO :      	('\tround 1: 0.06309077620506286\n'
INFO :      	 '\tround 2: 0.05520714672406515\n'
INFO :      	 '\tround 3: 0.051305106480916345\n')History (loss, centralized):
INFO :      	('\tround 0: 0.07384053850173951\n'
INFO :      	 '\tround 1: 0.06361047673225403\n'
INFO :      	 '\tround 2: 0.05634349513053894\n'
INFO :      	 '\tround 3: 0.053451504945755006\n')History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.1), (1, 0.258), (2, 0.35), (3, 0.37)]}
INFO :      


(ClientAppActor pid=3712) Epoch 1: train loss 0.053093209862709045, accuracy 0.37355555555555553 [repeated 3x across cluster]
(ClientAppActor pid=3713) [Client 8] evaluate, config: {} [repeated 2x across cluster]


INFO :      Terminated RayBackend
INFO :      Stopping Simulation Engine now.


## Sending/receiving arbitrary values to/from clients

In some situations, we want to configure client-side execution (training, evaluation) from the server-side. One example for that is the server asking the clients to train for a certain number of local epochs. Flower provides a way to send configuration values from the server to the clients using a dictionary. Let's look at an example where the clients receive values from the server through the `config` parameter in `fit` (`config` is also available in `evaluate`). The `fit` method receives the configuration dictionary through the `config` parameter and can then read values from this dictionary. In this example, it reads `server_round` and `local_epochs` and uses those values to improve the logging and configure the number of local training epochs:

In [11]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.cid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

So how can we  send this config dictionary from server to clients? The built-in Flower Strategies provide way to do this, and it works similarly to the way server-side evaluation works. We provide a function to the strategy, and the strategy calls this function for every round of federated learning:

In [12]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,  #
    }
    return config

Next, we'll just pass this function to the FedAvg strategy before starting the simulation:

In [13]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

# Create an instance of the ServerApp with the config and strategy defined above
server_app = fl.server.ServerApp(config=config, strategy=strategy)

# Start simulation
fl.simulation.run_simulation(
    server_app=server_app,
    client_app=client_app,
    backend_config=backend_config,
    num_supernodes=NUM_CLIENTS,
)

INFO :      Flower ECE: Starting Driver API (gRPC-rere) on 0.0.0.0:9091
INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      Registered 10 nodes
INFO :      
INFO :      Supported backends: ['ray']
INFO :      [INIT]
INFO :      Initialising: RayBackend
INFO :      Using initial global parameters provided by strategy
INFO :      Backend config: {'client_resources': None}
INFO :      DriverServicer.CreateRun
INFO :      Evaluating initial global parameters
INFO :      initial parameters (loss, other metrics): 0.07368363761901855, {'accuracy': 0.1}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.07368363761901855 / accuracy 0.1


2024-04-19 16:10:06,861	INFO worker.py:1621 -- Started a local Ray instance.
INFO :      `client_resources` not specified in backend config. Applying default setting: {'num_cpus': 2, 'num_gpus': 0.0}
INFO :      Constructed ActorPool with: 5 actors


(ClientAppActor pid=48001) [Client 1, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(ClientAppActor pid=48001) Epoch 1: train loss 0.06538806110620499, accuracy 0.2328888888888889


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      fit progress: (1, 0.06176188778877258, {'accuracy': 0.32}, 6.916007542051375)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


...

INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      fit progress: (3, 0.04994053840637207, {'accuracy': 0.428}, 23.670991208055057)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.04994053840637207 / accuracy 0.428


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 rounds in 27.10s
INFO :      History (loss, distributed):
INFO :      	('\tround 1: 0.061379447301228844\n'
INFO :      	 '\tround 2: 0.05247876977920533\n'
INFO :      	 '\tround 3: 0.04988230125109355\n')History (loss, centralized):
INFO :      	('\tround 0: 0.07368363761901855\n'
INFO :      	 '\tround 1: 0.06176188778877258\n'
INFO :      	 '\tround 2: 0.053729615688323976\n'
INFO :      	 '\tround 3: 0.04994053840637207\n')History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.1), (1, 0.32), (2, 0.388), (3, 0.428)]}
INFO :      


(ClientAppActor pid=47999) [Client 9] evaluate, config: {} [repeated 3x across cluster]
(ClientAppActor pid=48001) Epoch 2: train loss 0.048406489193439484, accuracy 0.42844444444444446 [repeated 3x across cluster]


INFO :      Terminated RayBackend
INFO :      Stopping Simulation Engine now.


As we can see, the client logs now include the current round of federated learning (which they read from the `config` dictionary). We can also configure local training to run for one epoch during the first and second round of federated learning, and then for two epochs during the third round.

Clients can also return arbitrary values to the server. To do so, they return a dictionary from `fit` and/or `evaluate`. We have seen and used this concept throughout this notebook without mentioning it explicitly: our `FlowerClient` returns a dictionary containing a custom key/value pair as the third return value in `evaluate`.

## Scaling federated learning

As a last step in this notebook, let's see how we can use Flower to experiment with a large number of clients.

In [ ]:
NUM_CLIENTS = 1000

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

We now have 1000 partitions, each holding 45 training and 5 validation examples. Given that the number of training examples on each client is quite small, we should probably train the model a bit longer, so we configure the clients to perform 3 local training epochs. We should also adjust the fraction of clients selected for training during each round (we don't want all 1000 clients participating in every round), so we adjust `fraction_fit` to `0.05`, which means that only 5% of available clients (so 50 clients) will be selected for training each round:


In [ ]:
def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config


strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.025,  # Train on 25 clients (each round)
    fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
    min_fit_clients=20,
    min_evaluate_clients=40,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    on_fit_config_fn=fit_config,
)

# Create an instance of the ServerApp with the config and strategy defined above
server_app = fl.server.ServerApp(config=config, strategy=strategy)

# Start simulation
fl.simulation.run_simulation(
    server_app=server_app,
    client_app=client_app,
    backend_config=backend_config,
    num_supernodes=NUM_CLIENTS,
)

## Recap

In this notebook, we've seen how we can gradually enhance our system by customizing the strategy, initializing parameters on the server side, choosing a different strategy, and evaluating models on the server-side. That's quite a bit of flexibility with so little code, right?

In the later sections, we've seen how we can communicate arbitrary values between server and clients to fully customize client-side execution. With that capability, we built a large-scale Federated Learning simulation using the Flower Virtual Client Engine and ran an experiment involving 1000 clients in the same workload - all in a Jupyter Notebook!

## Next steps

Before you continue, make sure to join the Flower community on Slack: [Join Slack](https://flower.ai/join-slack/)

There's a dedicated `#questions` channel if you need help, but we'd also love to hear who you are in `#introductions`!

The [Flower Federated Learning Tutorial - Part 3](https://flower.ai/docs/framework/tutorial-build-a-strategy-from-scratch-pytorch.html) shows how to build a fully custom `Strategy` from scratch.